<a href="https://colab.research.google.com/github/LizLian/from_scratch_2025/blob/main/Bigram_012026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Bigram Template

1. Read in the name.txt file, build a dictionary to store the bigram and their count from the dataset.
2. Sample from the model. Steps when encounters the end of sentence symbol.
3. write the negative log likelihood loss function. (cross entropy).
4. convert the bigram model to neural net. (hint: convert x to one hot vectors, initialize weights, logits = xenc @ W. Update W.data in backward pass)
5. sample the model again to see the results after optimization.

In [1]:
data = []
with open("names.txt") as infile:
  for line in infile:
    data.append(line.strip())

In [2]:
data[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [12]:
vocab = set(".".join(data))

In [41]:
stoi = {}
itos = {}
for i, c in enumerate(sorted(vocab)):
  stoi[c] = i
  itos[i] = c


In [42]:
bigrams = torch.zeros((len(vocab), len(vocab)))
for name in data:
  chrs = "." + name + "."
  for ch1, ch2 in zip(chrs, chrs[1:]):
    bigrams[stoi[ch1], stoi[ch2]] += 1

In [58]:
# @title Sample from the model
start_chr_prob = bigrams[0].float()/bigrams[0].sum()
start_chr_prob

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [75]:
import torch
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
  ix = 0
  out = []
  while True:
    p = bigrams[ix].float()/bigrams[ix].sum()
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    c = itos[ix]
    out.append(c)
    if ix == 0:
      break
  print("".join(out))

cexze.
momasurailezitynn.
konimittain.
llayn.
ka.


In [80]:
# @title Negative Log likelihood
nll = -torch.log(p).sum()
nll

tensor(109.1046)

In [97]:
import torch.nn.functional as F

xs = []
ys = []
for name in data:
  chrs = "." + name + "."
  for ch1, ch2 in zip(chrs, chrs[1:]):
    xs.append(stoi[ch1])
    ys.append(stoi[ch2])

xs = torch.tensor(xs)
num = xs.nelement()
ys = torch.tensor(ys)
W = torch.randn((27, 27), generator=g, requires_grad=True) # random select from a normal distribution (mean=0, std=1)

for i in range(100): # 50 epochs
  xenc = F.one_hot(xs, num_classes=27).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts/counts.sum(dim=1, keepdim=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  if i%10==0:
    print(loss.item())

  # backward pass
  W.grad = None
  loss.backward()

  # update
  W.data += -70 * W.grad


3.625509023666382
2.6165945529937744
2.5435054302215576
2.5182595252990723
2.5057497024536133
2.4984216690063477
2.493731737136841
2.4905762672424316
2.4883792400360107
2.486804962158203


In [98]:
# @title Model Generate Function
for i in range(5):
  ix = 0
  out = []
  while True:
    # p = bigrams[ix].float()/bigrams[ix].sum()
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    p = counts/counts.sum(dim=1, keepdim=True)
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    c = itos[ix]
    out.append(c)
    if ix == 0:
      break
  print("".join(out))

ckae.
a.
hidaruyia.
ezera.
duillpwghanamahu.
